In [1]:
ATTRIBUTE = 'sandbox'
PARALLEL = 3

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')
snorkel_postgres = os.environ['SNORKELDB'].startswith('postgres')
print snorkel_postgres

True


In [2]:
import os
if snorkel_postgres:
    os.environ['SNORKELDBNAME'] = ATTRIBUTE
    print os.system("dropdb " + os.environ['SNORKELDBNAME'])
    print os.system("createdb " + os.environ['SNORKELDBNAME'])
    print "SNORKELDBNAME = %s" % os.environ['SNORKELDBNAME']
else:
    try:
        os.remove('snorkel.db')
    except:
        pass

from snorkel import SnorkelSession
session = SnorkelSession()

0
0
SNORKELDBNAME = sandbox


In [3]:
# os.environ['SNORKELDBNAME'] = 'sandbox'
# os.system("dropdb " + os.environ['SNORKELDBNAME'])
# os.system("createdb " + os.environ['SNORKELDBNAME'])

In [4]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

# sandbox_path = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/sample/'
# file_name = 'RohmSemiconductor_SST6427T116'
# html_path = sandbox_path
# pdf_path = sandbox_path
sandbox_path = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/'
file_name = 'diseases'
html_path = sandbox_path + file_name + '.html'
pdf_path = sandbox_path
doc_parser = HTMLParser(path=html_path)
context_parser = OmniParser(tabular=True, 
                            lingual=True, blacklist=['style'], 
                            flatten=['span','br','i'], flatten_delim='',
                            visual=True, pdf_path=pdf_path)
#                             replacements=[('Temperature', 'Temp')])
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [5]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%
CPU times: user 168 ms, sys: 25.6 ms, total: 193 ms
Wall time: 531 ms
37 Phrases in Corpus (Sandbox)


In [6]:
for p in corpus.documents[0].phrases:
    print (p.page[0], p.table, p.text)

(1, None, u'Types of viruses, coughs, and colds')
(1, None, u'Here isa line break')
(1, None, u"I don't have Brain Canceror the hiccups")
(1, None, u'See Table 1 Below.')
(1, None, u'See Table 2 Below.')
(1, None, u'SeeTable 3Below.')
(1, None, u'Common Ailments')
(1, Table(Doc: diseases, Position: 0), u'Disease')
(1, Table(Doc: diseases, Position: 0), u'Location')
(1, Table(Doc: diseases, Position: 0), u'Year')
(1, Table(Doc: diseases, Position: 0), u'Polio and BC546 is -55OC cold.')
(1, Table(Doc: diseases, Position: 0), u'-Dublin to Milwaukee')
(1, Table(Doc: diseases, Position: 0), u'?')
(1, Table(Doc: diseases, Position: 0), u'2001')
(1, Table(Doc: diseases, Position: 1), u"I don't like TIPL761 or Chicken Pox or pizza. ")
(1, Table(Doc: diseases, Position: 1), u'Shingles is also bad.')
(1, Table(Doc: diseases, Position: 0), u'whooping cough')
(1, Table(Doc: diseases, Position: 0), u'Scurvy')
(1, Table(Doc: diseases, Position: 0), u'Annapolis')
(1, Table(Doc: diseases, Position: 0)

In [7]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
# Temp = candidate_subclass('Temp', ['temp'])
# Disease = candidate_subclass('Disease', ['disease'])
# Part = candidate_subclass('Part', ['part'])
Disease_Part = candidate_subclass('Disease_Part', ['disease','part'])

In [8]:
from snorkel.candidates import OmniNgrams
from hardware_spaces import OmniNgramsPart, OmniNgramsTemp

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)
omni_part = OmniNgrams(n_max=1)
# omni_part = OmniNgramsPart(n_max=1)
# omni_temp = OmniNgramsTemp(n_max=3)
# omni_temp = OmniNgrams(n_max=2)

In [9]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 'brain cancer', 'hiccups', 
            'ailments', 'disease', 'polio', 'chicken pox', 'shingles', 'whooping cough',
            'scurvy', 'infectious diseases', 'heart attack', 'arthritis', 'fever', 
            'yellow fever', 'hypochondria', 'pneumonia', 'warts']
print "Loaded %d diseases." % len(diseases)

Loaded 22 diseases.


In [10]:
from snorkel.matchers import *
from hardware_matchers import get_matcher

disease_dict_matcher = DictionaryMatch(d=diseases, ignore_case=True)
disease_matcher = disease_dict_matcher
# disease_len_matcher = LambdaFunctionMatch(func=lambda x: len(x.get_span().split()) > 1)
# disease_matcher = Intersect(disease_dict_matcher, disease_len_matcher)
# year_matcher = DateMatcher()
# part_matcher = RegexMatchEach(rgx='BC.*')
# temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]')
part_matcher = get_matcher('part')

In [11]:
from snorkel.candidates import CandidateExtractor

# ce = CandidateExtractor(Part, [omni_part], [part_matcher])
# ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
# ce = CandidateExtractor(Temp, [omni_temp], [temp_matcher])
ce = CandidateExtractor(Disease_Part, 
                        [omni_ngrams, omni_part], 
                        [disease_matcher, part_matcher],
                        throttler = None)

In [12]:
from snorkel.async_candidates import parallel_extract
candidates = parallel_extract(session, ce, corpus, 'Sandbox Candidates', parallel=PARALLEL)

[========================================] 100%


In [13]:
from snorkel.lf_helpers import *
for i, c in enumerate(candidates):
    print i, c
#     print c.disease.get_attrib_tokens('page')
#     print get_page_vert_percentile(c)
#     print get_page_horz_percentile(c)

0 Disease_Part(Span("colds", parent=5, chars=[30,34], words=[7,7]), Span("BC546", parent=44, chars=[10,14], words=[2,2]))
1 Disease_Part(Span("colds", parent=5, chars=[30,34], words=[7,7]), Span("TIPL761", parent=48, chars=[13,19], words=[4,4]))
2 Disease_Part(Span("Pneumonia", parent=40, chars=[27,35], words=[7,7]), Span("BC546", parent=44, chars=[10,14], words=[2,2]))
3 Disease_Part(Span("Pneumonia", parent=40, chars=[27,35], words=[7,7]), Span("TIPL761", parent=48, chars=[13,19], words=[4,4]))
4 Disease_Part(Span("heart attack", parent=12, chars=[47,58], words=[10,11]), Span("BC546", parent=44, chars=[10,14], words=[2,2]))
5 Disease_Part(Span("heart attack", parent=12, chars=[47,58], words=[10,11]), Span("TIPL761", parent=48, chars=[13,19], words=[4,4]))
6 Disease_Part(Span("Scurvy", parent=51, chars=[0,5], words=[0,0]), Span("BC546", parent=44, chars=[10,14], words=[2,2]))
7 Disease_Part(Span("Scurvy", parent=51, chars=[0,5], words=[0,0]), Span("TIPL761", parent=48, chars=[13,19], 

In [15]:
from snorkel.lf_helpers import *

for c in candidates:
    if not (c.part.get_span()=='TIPL761'): continue
    print c.disease
    print list(get_vert_ngrams(c.disease))
    print list(get_horz_ngrams(c.disease))

Span("colds", parent=5, chars=[30,34], words=[7,7])
[]
[]
Span("Pneumonia", parent=40, chars=[27,35], words=[7,7])
[u'Hypochondria', u'Table', u'Problem', u'Arthritis', u'YellowFever']
[u'and', u'how', u'much', u'they', u'cost', u'.']
Span("heart attack", parent=12, chars=[47,58], words=[10,11])
[]
[]
Span("Scurvy", parent=51, chars=[0,5], words=[0,0])
[u'Table', u'1', u':', u'.', u'Disease', u'BC546', u'is']
[u'and', u'to', u'-55', u'Annapolis', u'Junction', u'Storage', u'Temperature', u'-55', u'150', u'o', u'?', u'C']
Span("Hypochondria", parent=64, chars=[0,11], words=[0,0])
[u'Table', u'2', u':', u'Three', u'Pneumonia', u'and', u'Problem', u'Arthritis', u'YellowFever']
[u'Fear', u'100']
Span("Polio", parent=44, chars=[0,4], words=[0,0])
[]
[u'BC546', u'is', u'OC', u'cold', u'-', u'Dublin', u'Milwaukee', u'?', u'2001', u'and', u'to']
Span("coughs", parent=5, chars=[18,23], words=[4,4])
[]
[]
Span("Disease", parent=41, chars=[0,6], words=[0,0])
[u'Table', u'1', u':', u'.', u'BC546', 

In [ ]:
from snorkel.visualizer import Visualizer

print pdf_path
print file_name
pdf_file = pdf_path
viz = Visualizer(pdf_file)
viz.display_words(corpus.documents[0].phrases)

In [ ]:
for c in candidates:
    print c

In [ ]:
from snorkel.features.features import get_all_feats
c = candidates[4]
print c
print
for f, _ in get_all_feats(c):
    print f

In [ ]:
from snorkel.lf_helpers import *

s = c[0]
print s
print get_tag(s)
print get_parent_tag(s)
print get_prev_sibling_tags(s)
print get_prev_sibling_tags(s)
print get_next_sibling_tags(s)
print get_ancestor_class_names(s)
print get_ancestor_tag_names(s)
print get_ancestor_id_names(s)

In [ ]:
from snorkel.lf_helpers import *

# print [cell.text[:8] for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer)]
# print [phrase.text for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer) for phrase in cell.phrases]
print c.disease.parent
print c.part.parent
print list(get_left_ngrams(c.part, n_max=1, window=10, lower=False))
print list(get_right_ngrams(c.part, n_max=1, window=5))
print list(get_phrase_ngrams(c.part, n_max=1))
print list(get_cell_ngrams(c.part, n_max=1))
print list(get_row_ngrams(c.part, n_max=1, direct=True, infer=True))
print list(get_head_ngrams(c.part, 'row'))
print list(get_head_ngrams(c.part, 'col'))

In [ ]:
from snorkel.models import CandidateSet
train = CandidateSet(name='train', candidates=candidates[:25])
dev = CandidateSet(name='test', candidates=candidates[25:])

In [ ]:
from snorkel.fast_annotations import FeatureManager

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Sandbox Features')
F_train

# from snorkel.features import get_span_feats
# c = candidates[10]
# %prun for feat in get_span_feats(c): print feat

In [ ]:
from snorkel.utils import get_keys_by_candidate

c = candidates[1]
print c
print
for f in get_keys_by_candidate(c, F_train)[:]: print f

In [ ]:
# c = candidates[0]
# print c
# print c.year
# print c.year.parent
# print c.year.char_start
# print c.year.char_end
# print c.year.parent.words
# print c.year.parent.char_offsets
# print c.year.parent.lemmas
# print c.year.parent.pos_tags
# print c.year.parent.ner_tags
# print c.year.parent.dep_parents
# print c.year.parent.dep_labels
# print c.year.get_word_start()
# print c.year.get_word_end()
# # print get_row_ngrams(c.year)
# # print get_row_ngrams(c.year, infer=True)
# %time for i in range(1000): _get_aligned_cells((c.year).parent.cell, 'col', infer=True)

In [ ]:
# from snorkel.models import Span, ImplicitSpan, TemporarySpan

# print isinstance(c.year, TemporarySpan)

In [ ]:
# from snorkel.lf_helpers import get_between_ngrams, get_left_ngrams, get_right_ngrams
# from snorkel.lf_helpers import contains_token, contains_regex
# from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
# from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
# from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
# from snorkel.lf_helpers import _get_aligned_cells, _get_nonempty_cell

In [ ]:
from snorkel.lf_helpers import *
c = candidates[2]
print list(get_head_ngrams(c.disease, axis='col'))
# print get_left_ngrams(c.disease)
# print get_right_ngrams(c.disease)
# print contains_token(c, 'plague')
# print contains_regex(c, r'pla')
# print same_document(c)
# print same_table(c)
# print same_cell(c)
# print same_phrase(c)
# print get_phrase_ngrams(c.disease, n_min=1, n_max=3, case_sensitive=True)
# print get_cell_ngrams(c.disease, attrib='pos_tags')
# print get_neighbor_cell_ngrams(c.disease, dist=2, directions=True)
# print get_row_ngrams(c.disease)
# print get_col_ngrams(c.disease)
# print get_aligned_ngrams(c.disease)
# print get_aligned_ngrams(c.disease, infer=True)

In [ ]:
# from hardware_utils import expand_implicit_text 

# for part in expand_implicit_text(''.join(['BC856/857/858', '/', '859/860'])): print part

In [ ]:
# def foo(n):
#     phrase = 'repeat me'
#     pmul = phrase * n
#     pjoi = ''.join([phrase for x in xrange(n)])
#     pinc = ''
#     for x in xrange(n):
#         pinc += phrase
#     del pmul, pjoi, pinc

In [ ]:
# %lprun -f foo foo(1000)

In [ ]:
# from hardware_utils import get_gold_dict

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# gold_dict = get_gold_dict(filename, 'stg_temp_min')

In [ ]:
# print len(gold_dict.values())
# print gold_dict.values().count(1)

In [ ]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time (certain, maybe) = count_hardware_labels(loader, candidates, filename, attrib='stg_temp_min', attrib_class='temp')

In [ ]:
# %time 
# for i in range(10000): 
#     for j in range(100):
#         1 + 1

In [ ]:
# from snorkel.utils import ProgressBar;

In [ ]:
# %%time
# n = 100000
# pb = ProgressBar(n)
# for i in xrange(n):
#     pb.bar(i)
# pb.close()

In [ ]:
# N = 235
# N = 10
# ticks = set([int(i * N/100.0) for i in range(1,101)])
# print ticks

In [ ]:
for doc, text in doc_parser.parse():
    pass

In [ ]:
print text

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text, 'lxml')

In [ ]:
def create_xpath(tags, counts):
    xpath = ''
    for i, tag in enumerate(tags):
        xpath += '/' + tag
        if counts[i] != 1:
            xpath += '[%d]' % (counts[i] - 1)
    return xpath

In [ ]:
tags = ['html','body','table','tr','th']
counts = [1, 1, 2, 2, 1]
print create_xpath(tags, counts)

In [ ]:
print soup.html.body.table.next_element

In [ ]:
from lxml.html import fromstring
import lxml

html = lxml.html.fromstring(text)
# htmlparser = etree.HTMLParser()
# xml.etree.ElementTree.fromstring(text)
# tree = lxml.etree.fromstring(text, htmlparser)
# tree.xpath(xpathselector)

In [ ]:
etree.tostring(tree)

In [ ]:
print tree.xpath('/html/body/table[1]')
print tree.xpath('/html/body/table[1]')

In [ ]:
import Levenshtein
from editdistance import eval as editdist

In [ ]:
timeit editdist('blue','blackbird')

In [ ]:
timeit Levenshtein.distance('blue','blackbird')